# The Flashlight Kernel

In [1]:
def affinity_from_flow(flow, directions_array, flow_strength = 1, sigma=1):
    """Compute probabilities of transition in the given directions based on the flow. 

    Parameters
    ----------
    flow : torch tensor of shape n_points x n_dims
        _description_
    directions_array : torch tensor of shape n_directions x n_points x n_dims. Assumed to be normalized.
        _description_
    sigma : int, optional
        kernel bandwidth, by default 1
    returns (n_points)
    """
    assert len(flow.shape) == 2 # flow should only have one dimension
    assert len(directions_array.shape) > 1 and len(directions_array.shape) < 4
    n_directions = directions_array.shape[0]
    # Normalize directions
    length_of_directions = torch.linalg.norm(directions_array,dim=-1)
    normed_directions = F.normalize(directions_array,dim=-1)
    # and normalize flow # TODO: Perhaps reconsider
    # Calculate flow lengths, used to scale directions to flow
    # flow_lengths = torch.linalg.norm(flow,dim=-1)
    if len(directions_array) == 1: # convert to 2d array if necessary
      directions_array = directions_array[:,None] 
    # scale directions to have same norm as flow
    # scaled_directions = normed_directions * flow_lengths[:,None].repeat(directions_array.shape[0],1,directions_array.shape[2])
    # compute dot products as matrix multiplication
    dot_products = (normed_directions * flow).sum(-1)
    # take distance between flow projected onto direction and the direction
    distance_from_flow = (torch.linalg.norm(flow,dim=1)).repeat(n_directions,1) - dot_products
    # take absolute value
    distance_from_flow = torch.abs(distance_from_flow)
    # print('shape of dff',distance_from_flow.shape)
    # add to this the length of each direction
    distance_from_flow = flow_strength*distance_from_flow + length_of_directions
    # put the points on rows, directions in columns
    distance_from_flow = distance_from_flow.T
    # take kernel of distances
    kernel =  torch.exp(-distance_from_flow/sigma)
    
    return kernel

In [2]:
def affinity_matrix_from_pointset_to_pointset(pointset1, pointset2, flow,n_neighbors=None,sigma=0.5, flow_strength=1):
    """Compute affinity matrix between the points of pointset1 and pointset2, using the provided flow.

    Parameters
    ----------
    pointset1 : torch tensor, n1 x d
        The first pointset, to calculate affinities *from*
    pointset2 : torch tensor, n2 x d
        The second pointset, to calculate affinities *to* (from pointset1)
    flow : a function that, when called at a point, gives the flow at that point
    n_neighbors : number of neighbors to include in affinity computations. All neighbors beyond it are given affinity zero
    (currently not implemented)

    Returns:
    Affinity matrix: torch tensor of shape n1 x n2
    """
    
    # Calculate the directions from point i in pointset 1 to point j in pointset 2
    n1 = pointset1.shape[0]
    n2 = pointset2.shape[0]
    P2 = pointset2[:,:,None].repeat(1,1,n1)
    P1 = pointset1.T.repeat(n2,1,1)
    P3 = (P2 - P1)
    P3 = P3.transpose(1,2)
    # dimension 1 represents directions to point i
    # dimension 2 represents direction from point j
    # dimension 3 represents direction in each dimension (R^n)
    # compute affinities from flow and directions
    affinities = affinity_from_flow(flow,P3,sigma=sigma,flow_strength=flow_strength)
    
    return affinities